In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\nihal\AppData\Local\Temp/ipykernel_14768/1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\nihal\AppData\Local\Temp/ipykernel_14768/1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
import wget
spacex_launch_dash=wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


100% [................................................................................] 2476 / 2476

In [11]:
spacex_df.dtypes

Unnamed: 0                    int64
Flight Number                 int64
Launch Site                  object
class                         int64
Payload Mass (kg)           float64
Booster Version              object
Booster Version Category     object
dtype: object

In [5]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:



# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',  options=[{'label': 'All Sites', 'value': 'All Sites'},{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                                           {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                                           {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                                           {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                            value='All Sites',placeholder='Select a Launch Site here', searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                                marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
             Input(component_id='site-dropdown',component_property='value'))
def pie_chart(launch_site):
    if launch_site=='All Sites':
        success_data=spacex_df[spacex_df['class']==1]
        y=success_data.groupby('Launch Site')['class'].sum().reset_index()
        fig=px.pie(values=y['class'], names=y['Launch Site'], title='Total success of each launch site')
    else:
        fig=px.pie(values=spacex_df[spacex_df['Launch Site']==str(launch_site)]['class'].value_counts(normalize=True), 
                  names=spacex_df['class'].unique(), 
                  title='Total Success Launches for Site '+str(launch_site))
    return(fig)
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown',component_property='value'), Input(component_id='payload-slider', component_property='value')])
def scatter_chart(site, payload):
    if site=='All Sites':
        fig=px.scatter(spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])],
                      x='Payload Mass (kg)', y='Launch Site', color='class',hover_data=['Launch Site'])
    else:
        site_data=spacex_df[spacex_df['Launch Site']==site]
        payload_data=site_data[site_data['Payload Mass (kg)'].between(payload[0],payload[1])]
        fig=px.scatter(payload_data,x='Payload Mass (kg)', y='class',hover_data=['Launch Site'])
    return(fig)

# Run the app
if __name__ == '__main__':
    app.run_server(port=8052)
